# Processing Experimental Data
This file takes the raw experimental data at the four signal levels we studied and produces approximates for the training and testing portions of the datasets.

In [ ]:
import h5py
import numpy as np
import torch as t
from RPI_tools import pytorch_tools
from tqdm import tqdm
from scipy import io

# Edit this line to point to the location of the data folder on your system.
data_folder_prefix = 'data/'

In [ ]:
# ABE - I added "1000" to the list here, before it was missing

photon_levels = [1, 10, 100, 1000]
for photon in photon_levels:
    
    print('Processing data for target photon level %d.' % photon)

    # We load the raw patterns from the experimental datasets
    filename = data_folder_prefix + ('Experimental/RPI_Datasets/Collection_3_photon_%d.npy' % photon)
    
    patterns = np.load(filename)
    print(np.shape(patterns))
    
    # The first 4,000 images are used as the training data
    train = patterns[:4000] # ABE - This was originally [:4] - was that a typo?
    # And the next 100 images are the test dataset.
    test = patterns[4000:4100]
    
    # Now, we load the appropriate probe function, as produced from a separate ptychography scan.
    calibration_data = io.loadmat(data_folder_prefix + 
                                  ('Experimental/Ptychography_Calibrations/Calibrated_Ptychography_Result_EMGain3800_%dPhotonTarget.mat' % photon))
    expanded_probe = calibration_data['probe'][0]
    background = calibration_data['background']

    print('Dataset and calibration data loaded.')
    
    lr = 1
    iters = 1

    # The iterative algorithm is defined in pytorch for historical reasons. This is the reason that
    # this codebase depends on both pytorch and tensorflow.
    expanded_probe0 = t.from_numpy(expanded_probe)
    background = t.from_numpy(background)
    train_patterns = t.from_numpy(train)
    test_patterns = t.from_numpy(test)

    # Here, we run one iteration of the automatic-differentiation RPI algorithm to produce the training approximants
    train_approx = []
    for pattern in tqdm(train_patterns):
        result, _ = pytorch_tools.reconstruct(pattern, expanded_probe0, 256, lr, iters, background=background, 
                                              loss_func=pytorch_tools.amplitude_mse, GPU=True, saturation=2**14)
        # This accounts for a 90 degree rotation between the definition of the images and the detector orientation
        train_approx.append(np.rot90(result.numpy(), 2))
        
    print('Training approximants produced.')

    # Here, we run one iteration of the automatic-differentiation RPI algorithm to produce the testing approximants
    test_approx = []
    for pattern in test_patterns:
        result, _ = pytorch_tools.reconstruct(pattern, expanded_probe0, 256, lr, iters, background=background, 
                                              loss_func=pytorch_tools.amplitude_mse, GPU=True, saturation=2**14)
        # This accounts for a 90 degree rotation between the definition of the images and the detector orientation
        test_approx.append(np.rot90(result.numpy(), 2))

    print('Testing approximants produced.')
    
    # The approximant is defined to just be the phase channel of the output.
    train_approx = np.angle(np.array(train_approx))
    test_approx = np.angle(np.array(test_approx))

    # Finally, we save out the approximants that we've created
    np.save('train-approx-%d-iter-%d-lr-%0.2f.npy' % (photon, iters, lr), train_approx)
    np.save('test-approx-%d-iter-%d-lr-%0.2f.npy' % (photon, iters, lr), test_approx)
    
print('All approximants saved, have a nice day!')

In [ ]:
from matplotlib import pyplot as plt

Xs, Ys = np.mgrid[:256,:256]
Xs = Xs - np.mean(Xs)
Ys = Ys - np.mean(Ys)
Rs = np.sqrt(Xs**2 + Ys**2)

photon_levels = [1, 10, 100, 1000]
for photon in photon_levels:
    test_approx = np.load('test-approx-%d-iter-%d-lr-%0.2f.npy' % (photon, iters, lr))
    test_approx[:, Rs>128] = 0
    
    plt.rcParams['figure.figsize'] = [20, 20]
    fig = plt.figure()
    fig.subplots_adjust(hspace=.05, wspace=.05)

    for i in range(16):
        ax = fig.add_subplot(4, 4, i +1)
        ax.axes.xaxis.set_visible(False)
        ax.axes.yaxis.set_visible(False)
        plt.imshow(test_approx[i])

In [ ]:
# Get 100 iteration reconstruction results

photon_levels = [1, 10, 100, 1000]
for photon in photon_levels:
    
    print('Processing data for target photon level %d.' % photon)

    # We load the raw patterns from the experimental datasets
    filename = data_folder_prefix + ('Experimental/RPI_Datasets/Collection_3_photon_%d.npy' % photon)
    
    patterns = np.load(filename)
    print(np.shape(patterns))
    
    # get the test dataset.
    test = patterns[4000:4100]
    
    # Now, we load the appropriate probe function, as produced from a separate ptychography scan.
    calibration_data = io.loadmat(data_folder_prefix + 
                                  ('Experimental/Ptychography_Calibrations/Calibrated_Ptychography_Result_EMGain3800_%dPhotonTarget.mat' % photon))
    expanded_probe = calibration_data['probe'][0]
    background = calibration_data['background']

    print('Dataset and calibration data loaded.')
    
    lr = 0.5
    iters = 1000

    # The iterative algorithm is defined in pytorch for historical reasons. This is the reason that
    # this codebase depends on both pytorch and tensorflow.
    expanded_probe0 = t.from_numpy(expanded_probe)
    background = t.from_numpy(background)
    test_patterns = t.from_numpy(test)

    # Here, we run one iteration of the automatic-differentiation RPI algorithm to produce the testing approximants
    test_approx = []
    for pattern in test_patterns:
        
        result, _ = pytorch_tools.reconstruct(pattern, expanded_probe0, 256, lr, iters, background=background, 
                                                  loss_func=pytorch_tools.amplitude_mse, GPU=True, saturation=2**14)
        # This accounts for a 90 degree rotation between the definition of the images and the detector orientation
        test_approx.append(np.rot90(result.numpy(), 2))

    print('Testing iterative reconstructions produced.')
    
    # The approximant is defined to just be the phase channel of the output.
    test_approx = np.angle(np.array(test_approx))

    # Finally, we save out the approximants that we've created
    np.save('test-approx-%d-iter-%d-lr-%0.2f.npy' % (photon, iters, lr), test_approx)
    
print('All approximants saved, have a nice day!')

In [ ]:
from matplotlib import pyplot as plt

Xs, Ys = np.mgrid[:256,:256]
Xs = Xs - np.mean(Xs)
Ys = Ys - np.mean(Ys)
Rs = np.sqrt(Xs**2 + Ys**2)

photon_levels = [1, 10, 100, 1000]
for photon in photon_levels:
    
    test_approx = np.load('test-approx-%d-iter-%d-lr-%0.2f.npy' % (photon, iters, lr))
    test_approx[:, Rs>128] = 0
    
#     if photon == 1e3:
#         test_approx = np.load('test-approx-%d-iter-%d-lr-%0.2f.npy' % (photon, iters, 1))
#         test_approx[:, Rs>128] = 0
    
    plt.rcParams['figure.figsize'] = [20, 20]
    fig = plt.figure()
    fig.subplots_adjust(hspace=.05, wspace=.05)

    for i in range(16):
        ax = fig.add_subplot(4, 4, i +1)
        ax.axes.xaxis.set_visible(False)
        ax.axes.yaxis.set_visible(False)
        plt.imshow(test_approx[i])

In [ ]:
R = 0.5

photon_levels = [1, 10, 100, 1000]
for photon in photon_levels:
    
    print('Processing E2E data for target photon level %d.' % photon)

    # We load the raw patterns from the experimental datasets
    filename = data_folder_prefix + ('Experimental/RPI_Datasets/Collection_3_photon_%d.npy' % photon)
    
    patterns = np.load(filename)
    print(np.shape(patterns))
    
    # The first 4,000 images are used as the training data
    train = patterns[:4000] # ABE - This was originally [:4] - was that a typo?
    # And the next 100 images are the test dataset.
    test = patterns[4000:4100]
    
    # Now, we load the appropriate probe function, as produced from a separate ptychography scan.
    calibration_data = io.loadmat(data_folder_prefix + 
                                  ('Experimental/Ptychography_Calibrations/Calibrated_Ptychography_Result_EMGain3800_%dPhotonTarget.mat' % photon))
    expanded_probe = calibration_data['probe'][0]
    background = calibration_data['background']
    
    # remove the background for experimental data
    tr_patterns = []
    for img in train:
        img -= background
        tr_patterns.append(img)
    tr_patterns = np.array(tr_patterns)
    # zero pad the data to (4000, 1024, 1024)
    tr_patterns = np.pad(tr_patterns, ((0, 0), (16, 16), (16, 16)))
    
    # remove the background for experimental data
    test_patterns = []
    for img in train:
        img -= background
        test_patterns.append(img)
    test_patterns = np.array(test_patterns)
    # zero pad the data to (100, 1024, 1024)
    test_patterns = np.pad(test_patterns, ((0, 0), (16, 16), (16, 16)))
    
    probe = np.pad(expanded_probe, ((16, 16), (16, 16)))
    
    chn = 4
    
    flatten = np.zeros((4000, 256, 256, chn**2)).astype(np.float32)

    for i in range(4000):
        for j in range(chn):
            for k in range(chn):
                flatten[i, :, :, chn*j + k] = tr_patterns[i, 256*j:256*(j+1), 256*k:256*(k+1)]
                
    flatten = flatten.astype(np.float32)

    np.save('exp-tr_patterns-flatten-R-%0.2f-phperpix-%d.npy' % (R, photon), flatten)
    
    
    flatten = np.zeros((100, 256, 256, chn**2))

    for i in range(100):
        for j in range(chn):
            for k in range(chn):
                flatten[i, :, :, chn*j + k] = test_patterns[i, 256*j:256*(j+1), 256*k:256*(k+1)]
                
    flatten = flatten.astype(np.float32)

    np.save('exp-test_patterns-flatten-R-%0.2f-phperpix-%d.npy' % (R, photon), flatten)
    

    flatten = np.zeros((256, 256, 2*chn**2))

    for j in range(chn):
        for k in range(chn):
            flatten[:, :, chn*j + k] = np.abs(probe[256*j:256*(j+1), 256*k:256*(k+1)])
            flatten[:, :, chn**2 + chn*j + k] = np.angle(probe[256*j:256*(j+1), 256*k:256*(k+1)])
            
    flatten = flatten.astype(np.float32)

    np.save('exp-probe-flatten-R-%0.2f-phperpix-%d.npy' % (R, photon), flatten)